In [ ]:
import torch
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your (x^1)
    [0.55, 0.87, 0.66], # journey (x^2)
    [0.57, 0.85, 0.64], # starts (x^3)
    [0.22, 0.58, 0.33], # with (x^4)
    [0.77, 0.25, 0.10], # one (x^5)
    [0.05, 0.80, 0.55]] # step (x^6)
)

In [ ]:
query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])
for i in range(len(inputs)):
    attn_scores_2[i]=torch.dot(query, inputs[i])
print(attn_scores_2)
# same as vec mat mul
t = torch.transpose(inputs,0,1)
inputs[1]@t

In [ ]:
# Simple normalization 
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
print(attn_weights_2_tmp.sum())

# More common to use softmax to normalize
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)
attn_weights_2_naive = softmax_naive(attn_scores_2)
print(attn_weights_2_naive)
print(attn_weights_2_naive.sum())

# better to use optimized torch softmax
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

In [ ]:
# create context vector by multiplying each input by its attention weight
query = inputs[1] # 2nd input token is the query
context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i
print(context_vec_2)

In [ ]:
# Now follow the same steps but for each input embedding rather than just one
input_length = inputs.shape[0]
embedding_length = inputs.shape[1]
attn_scores = torch.empty(input_length, input_length)
for i in range(input_length):
    for j in range(input_length):
        attn_scores[i,j] = torch.dot(inputs[i], inputs[j])

# or just use matrix multiplication 
attn_scores = inputs @ inputs.T

In [ ]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)
print(inputs)

In [ ]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

In [ ]:
# TRAINABLE ATTENTION ON SINGLE INPUT 
# computing only z2 (context vector for 2nd input)

x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)



In [ ]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(query_2)
W_key


In [ ]:
# calculation for just second input requires key value matrices for rest of the inputs
keys = inputs @ W_key
values = inputs @ W_value
print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

In [ ]:
keys_2 = keys[1]
print(keys_2)
attn_score_22 = query_2.dot(keys_2)
attn_score_22

In [ ]:
# for all other inputs, attn score relative to x[1] is done by dot of the Wk of the target input and the Wq of x[1]
attn_scores_2 = query_2 @ keys.T
attn_scores_2

In [ ]:
# go attn scores to attn weights by softmax
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

In [ ]:
context_vec_2  = attn_weights_2 @ values
context_vec_2 

In [ ]:
queries = inputs @ W_query
queries

In [ ]:
import torch.nn as nn   
class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))
            
    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec

In [ ]:
torch.manual_seed(789)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

In [ ]:
import torch.nn as nn   
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
            
    def forward(self, x):
        keys =  self.W_key(x)
        queries =  self.W_query(x)
        values =  self.W_value(x)
        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec

In [ ]:
torch.manual_seed(789)
sa_v2= SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))   
print(list(sa_v2.parameters()))
print(list(sa_v1.parameters()))


In [ ]:
# Linear intializes weights using a more efficient scheme, but copying shows the two models have same behavior
sa1 = SelfAttention_v1(d_in, d_out)
sa2 = SelfAttention_v2(d_in, d_out)
sa1.W_query.data=torch.t(sa2.W_query.weight)
sa1.W_value.data=torch.t(sa2.W_value.weight)
sa1.W_key.data=torch.t(sa2.W_key.weight)
print(sa1(inputs))
print(sa2(inputs))
